<a href="https://colab.research.google.com/github/deepak1195/NaturalLanguageProcessing/blob/main/013_FakeNewsClassification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import spacy

In [2]:
!python -m spacy download en_core_web_lg

2023-07-30 16:54:38.228994: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 587.7/587.7 MB 1.8 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_lg')


In [3]:
import numpy as np
import pandas as pd

In [4]:
from google.colab import drive
drive.mount('/content/drive')
from numpy.ma.core import shape
path="/content/drive/MyDrive/myWork/Data/"

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [5]:
df=pd.read_csv(f'{path}Fake_Real_Data.csv')
df

,Text,label
0,Top Trump Surrogate BRUTALLY Stabs Him In The...,Fake
1,U.S. conservative leader optimistic of common ...,Real
2,"Trump proposes U.S. tax overhaul, stirs concer...",Real
3,Court Forces Ohio To Allow Millions Of Illega...,Fake
4,Democrats say Trump agrees to work on immigrat...,Real
...,...,...
9895,Wikileaks Admits To Screwing Up IMMENSELY Wit...,Fake
9896,Trump consults Republican senators on Fed chie...,Real
9897,Trump lawyers say judge lacks jurisdiction for...,Real
9898,WATCH: Right-Wing Pastor Falsely Credits Trum...,Fake


In [6]:
df['eLabel']=df['label'].replace({'Fake':1,'Real':0})
df

,Text,label,eLabel
0,Top Trump Surrogate BRUTALLY Stabs Him In The...,Fake,1
1,U.S. conservative leader optimistic of common ...,Real,0
2,"Trump proposes U.S. tax overhaul, stirs concer...",Real,0
3,Court Forces Ohio To Allow Millions Of Illega...,Fake,1
4,Democrats say Trump agrees to work on immigrat...,Real,0
...,...,...,...
9895,Wikileaks Admits To Screwing Up IMMENSELY Wit...,Fake,1
9896,Trump consults Republican senators on Fed chie...,Real,0
9897,Trump lawyers say judge lacks jurisdiction for...,Real,0
9898,WATCH: Right-Wing Pastor Falsely Credits Trum...,Fake,1


In [7]:
nlp=spacy.load('en_core_web_lg')

In [8]:
df['eText']=df['Text'].apply(lambda x:nlp(x).vector)
df

,Text,label,eLabel,eText
0,Top Trump Surrogate BRUTALLY Stabs Him In The...,Fake,1,"[-0.6759837, 1.4263071, -2.318466, -0.451093, ..."
1,U.S. conservative leader optimistic of common ...,Real,0,"[-1.8355803, 1.3101058, -2.4919677, 1.0268308,..."
2,"Trump proposes U.S. tax overhaul, stirs concer...",Real,0,"[-1.9851209, 0.14389805, -2.4221718, 0.9133005..."
3,Court Forces Ohio To Allow Millions Of Illega...,Fake,1,"[-2.7812982, -0.16120885, -1.609772, 1.3624227..."
4,Democrats say Trump agrees to work on immigrat...,Real,0,"[-2.2010763, 0.9961637, -2.4088492, 1.128273, ..."
...,...,...,...,...
9895,Wikileaks Admits To Screwing Up IMMENSELY Wit...,Fake,1,"[-1.6682401, 0.78006977, -2.2337353, -0.159771..."
9896,Trump consults Republican senators on Fed chie...,Real,0,"[-1.9297235, 0.8007302, -1.8990824, 0.42668718..."
9897,Trump lawyers say judge lacks jurisdiction for...,Real,0,"[-1.5289013, 1.0250993, -1.9861357, 0.4278564,..."
9898,WATCH: Right-Wing Pastor Falsely Credits Trum...,Fake,1,"[-1.3928099, 0.7792715, -2.2072845, 0.13192406..."


In [9]:
from sklearn.model_selection import train_test_split
xTrain,xTest,yTrain,yTest=train_test_split(df.eText, df.eLabel, test_size=0.25, random_state=2023, stratify=df.eLabel)

In [10]:
xTrain

4834    [-1.5371963, 1.0873641, -2.3564248, 0.7811601,...
126     [-1.0232946, 0.53370005, -1.3557348, 0.0318621...
4022    [-0.5485656, 1.1053947, -2.3969123, -0.3442680...
7568    [-1.097689, 0.9017304, -1.6890976, 0.16017403,...
6250    [-1.8445455, 0.51682234, -1.4612978, 0.9416194...
                              ...                        
5314    [-1.9849584, 1.066484, -1.9978613, 0.69795233,...
9606    [-2.0159066, 1.1500455, -1.9120783, 0.21450247...
6738    [-2.1526146, 1.0074974, -1.9310542, 0.40111354...
4128    [-1.8167452, 1.037187, -2.4864714, 0.2803641, ...
9250    [-1.9071028, 0.7893087, -2.1176634, 0.70379275...
Name: eText, Length: 7425, dtype: object

In [11]:
xTrain2D=np.stack(xTrain)
xTest2D=np.stack(xTest)

In [12]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.preprocessing import MinMaxScaler
scaler=MinMaxScaler()

xTrain2DS=scaler.fit_transform(xTrain2D)
xTest2DS=scaler.fit_transform(xTest2D)

model=MultinomialNB()
model.fit(xTrain2DS,yTrain)

MultinomialNB()

In [13]:
yPred=model.predict(xTest2DS)
yPred

array([1, 0, 1, ..., 1, 1, 1])

In [14]:
from sklearn.metrics import classification_report
print(classification_report(yTest,yPred))

              precision    recall  f1-score   support

           0       0.96      0.94      0.95      1225
           1       0.94      0.96      0.95      1250

    accuracy                           0.95      2475
   macro avg       0.95      0.95      0.95      2475
weighted avg       0.95      0.95      0.95      2475



In [15]:
from sklearn.neighbors import KNeighborsClassifier
kModel=KNeighborsClassifier(n_neighbors=5,metric='euclidean')
kModel.fit(xTrain2DS,yTrain)
print(classification_report(yTest,kModel.predict(xTest2DS)))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00      1225
           1       1.00      1.00      1.00      1250

    accuracy                           1.00      2475
   macro avg       1.00      1.00      1.00      2475
weighted avg       1.00      1.00      1.00      2475

